In [228]:
import pandas as pd
import numpy as np

In [229]:
path = '/home/gas/Documents/Yandex_ML_track_2018/'

In [230]:
train = pd.read_csv(path+'data/train.tsv', sep='\t', quoting=3, header=None)
test = pd.read_csv(path+'data/public.tsv', sep='\t', quoting=3, error_bad_lines=False, header=None)

In [231]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 97533 entries, 0 to 97532
Data columns (total 8 columns):
0    97533 non-null int64
1    48559 non-null object
2    72502 non-null object
3    97533 non-null object
4    97533 non-null int64
5    97533 non-null object
6    97533 non-null object
7    97533 non-null float64
dtypes: float64(1), int64(2), object(5)
memory usage: 6.0+ MB


In [232]:
train.fillna('', inplace=True)
test.fillna('', inplace=True)

In [233]:
# расставляем значения по формуле хорошесть * confidence
# Good (3)
# Neutral (2)
# Bad (1)

In [234]:
def rank2num(st):
    if st == 'good':
        return 3
    else:
        if st == 'neutral':
            return 2
        else:
            return 1

In [235]:
train['rank'] = train[6].apply(rank2num)
train['target'] = train['rank'] * train[7]
train.head()

,0,1,2,3,4,5,6,7,rank,target
0,22579918886,"кликни на меня а потом на надпись "" видео - зв...","о , я тебя вижу .","ладно , повесь трубку .",0,не могу .,good,0.875352,3,2.626055
1,22579918886,"кликни на меня а потом на надпись "" видео - зв...","о , я тебя вижу .","ладно , повесь трубку .",1,"нет , звонить буду я .",neutral,0.900968,2,1.801936
2,22579918886,"кликни на меня а потом на надпись "" видео - зв...","о , я тебя вижу .","ладно , повесь трубку .",2,"слушай , я не мог уйти .",bad,0.884320,1,0.884320
3,22579918886,"кликни на меня а потом на надпись "" видео - зв...","о , я тебя вижу .","ладно , повесь трубку .",3,я не прекращу звонить .,good,0.982530,3,2.947591
4,22579918886,"кликни на меня а потом на надпись "" видео - зв...","о , я тебя вижу .","ладно , повесь трубку .",4,я звоню им .,good,0.838054,3,2.514161


In [15]:
test.head()

,0,1,2,3,4,5
0,138920940977,"знаешь , я иногда подумываю , что тебе надо пр...",не - а .,нет ?,0,неа .
1,138920940977,"знаешь , я иногда подумываю , что тебе надо пр...",не - а .,нет ?,1,"нет , не хочу ."
2,138920940977,"знаешь , я иногда подумываю , что тебе надо пр...",не - а .,нет ?,2,нет .
3,138920940977,"знаешь , я иногда подумываю , что тебе надо пр...",не - а .,нет ?,3,"конечно , нет ."
4,138920940977,"знаешь , я иногда подумываю , что тебе надо пр...",не - а .,нет ?,4,"разумеется , нет ."


In [16]:
import nltk
#nltk.download('wordnet')

from collections import OrderedDict
from gensim.utils import deaccent
import operator
import pickle

class getTokens:
    
    def __init__(self, garbage=['hello', 'hi', 'need', 'help', 'com', 'http', 'please', 'someone', 'etc', 'www']):
        self.tokenizer = nltk.RegexpTokenizer('\w+')        
        self.lmtzr = nltk.stem.wordnet.WordNetLemmatizer()
        self.garbage = set(garbage)
    
    def transform(self, text):
        text = self.tokenizer.tokenize(text)
        text = [self.lmtzr.lemmatize(deaccent(word.lower())) for word in text]
        text = [word for word in text if len(word) > 1 and word not in self.garbage]
        return ' '.join(text)

/home/gas/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [17]:
tokenize = getTokens()

In [18]:
text_cols = [1, 2, 3, 5]

In [19]:
for col in text_cols:
    train[col] = train[col].apply(tokenize.transform)
    test[col] = test[col].apply(tokenize.transform)

In [20]:
train.head()

,0,1,2,3,4,5,6,7,rank,target
0,22579918886,кликни на меня потом на надпись видео звонок,тебя вижу,ладно повесь трубку,0,не могу,good,0.875352,3,2.626055
1,22579918886,кликни на меня потом на надпись видео звонок,тебя вижу,ладно повесь трубку,1,нет звонить буду,neutral,0.900968,2,1.801936
2,22579918886,кликни на меня потом на надпись видео звонок,тебя вижу,ладно повесь трубку,2,слушаи не мог уити,bad,0.884320,1,0.884320
3,22579918886,кликни на меня потом на надпись видео звонок,тебя вижу,ладно повесь трубку,3,не прекращу звонить,good,0.982530,3,2.947591
4,22579918886,кликни на меня потом на надпись видео звонок,тебя вижу,ладно повесь трубку,4,звоню им,good,0.838054,3,2.514161


In [21]:
test.head()

,0,1,2,3,4,5
0,138920940977,знаешь иногда подумываю что тебе надо принести...,не,нет,0,неа
1,138920940977,знаешь иногда подумываю что тебе надо принести...,не,нет,1,нет не хочу
2,138920940977,знаешь иногда подумываю что тебе надо принести...,не,нет,2,нет
3,138920940977,знаешь иногда подумываю что тебе надо принести...,не,нет,3,конечно нет
4,138920940977,знаешь иногда подумываю что тебе надо принести...,не,нет,4,разумеется нет


In [22]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn import pipeline

In [196]:
svd_n_components = 300
max_features = 500000

In [212]:
counters_pipe = pipeline.FeatureUnion(
    n_jobs = -1,
    transformer_list = [
    ('chars_features', pipeline.Pipeline([
        ('chars_counter', CountVectorizer(
            analyzer=u'char', ngram_range=(2, 4), tokenizer=None,
            max_features=max_features, strip_accents=None, max_df=0.9, min_df=2, lowercase=False)),
        ('chars_tsvd', TruncatedSVD(n_components=svd_n_components, n_iter=25, random_state=42))])),
    ('words_features', pipeline.Pipeline([
        ('words_counter', CountVectorizer(
            analyzer=u'word', ngram_range=(1, 3), tokenizer=None,
            max_features=max_features, strip_accents=None, max_df=0.9, min_df=2, lowercase=False)),
        ('words_tsvd', TruncatedSVD(n_components=svd_n_components, n_iter=25, random_state=42))])),
])

tfidf_pipe = pipeline.Pipeline([
    ('words_tfidf', TfidfVectorizer(
       analyzer=u'word', ngram_range=(1, 3), tokenizer=None,
       max_features=max_features, strip_accents=None, max_df=0.9, min_df=2, lowercase=False)),
    ('words_tsvd', TruncatedSVD(n_components=svd_n_components*2, n_iter=25, random_state=42))])

In [ ]:
from gensim.models import KeyedVectors

skipgram_ru = KeyedVectors.load_word2vec_format(path+'/models/ruwikiruscorpora_upos_skipgram_300_2_2018.vec.gz')
lmtzr = nltk.stem.wordnet.WordNetLemmatizer()
new_skipgram_ru = {}
for word in skipgram_ru.vocab.keys():
    new_word = lmtzr.lemmatize(deaccent(word.split('_')[0]))
    new_skipgram_ru[new_word] = skipgram_ru[word]
del skipgram_ru

tfidf = TfidfVectorizer(
       analyzer=u'word', ngram_range=(1, 3), tokenizer=None, use_idf=True,
       max_features=10000, strip_accents=None, max_df=0.9, min_df=2, lowercase=False)

def weighter(transformed, tfidf, new_skipgram_ru):
    data = None
    for i in range(transformed.shape[0]):
        features = 0
        for j, word in enumerate(tfidf.vocabulary_):
            if word in new_skipgram_ru.keys():
                features += new_skipgram_ru[word] * transformed[i, tfidf.vocabulary_[word]]
        if data is None:
            data = features
        else:
            data = np.vstack([data, features])
    return data

In [116]:
def dcg_at_k(r, k, method=0):
    r = np.asfarray(r)[:k]
    if r.size:
        if method == 0:
            return r[0] + np.sum(r[1:] / np.log2(np.arange(2, r.size + 1)))
        elif method == 1:
            return np.sum(r / np.log2(np.arange(2, r.size + 2)))
        else:
            raise ValueError('method must be 0 or 1.')
    return 0.

def ndcg_at_k(r, k=3, method=0):
    dcg_max = dcg_at_k(sorted(r, reverse=True), k, method)
    if not dcg_max:
        return 0.
    return dcg_at_k(r, k, method) / dcg_max

In [53]:
from sklearn.linear_model import Ridge
from sklearn.model_selection import KFold

In [113]:
nfolds = round(len(train) / len(test))

lr = Ridge(alpha=3.0)
cv = KFold(n_splits=nfolds, shuffle=True, random_state=42)

In [114]:
fold = 1
cv_score = 0
for train_index, test_index in cv.split(train):
    print('make features...')
#     counters_pipe.fit(np.hstack([train[col].loc[train_index] for col in text_cols]))
#     train_data = np.hstack([counters_pipe.transform(train[col].loc[train_index]) for col in text_cols])
#     test_data = np.hstack([counters_pipe.transform(train[col].loc[test_index]) for col in text_cols])
    
    tfidf.fit(np.hstack([train[col].loc[train_index] for col in text_cols]))
    train_data = np.hstack([weighter(tfidf.transform(train[col].loc[train_index]), tfidf, new_skipgram_ru) for col in text_cols])
    test_data = np.hstack([weighter(tfidf.transform(train[col].loc[test_index]), tfidf, new_skipgram_ru) for col in text_cols])
    
    print('fitting model...')
    lr.fit(train_data, train['target'].loc[train_index])

    print('make prediction...')
    prediction = pd.DataFrame()
    prediction['context_id'] = train[0].loc[test_index]
    prediction['reply_id'] = train[4].loc[test_index]
    prediction['rank'] = - lr.predict(test_data)
    prediction = prediction.sort_values(by=['context_id', 'rank'])[['context_id', 'reply_id']]
    
    score = 0
    uniques = prediction['context_id'].unique()
    for Id in uniques:
        tmp = prediction[prediction['context_id'] == Id]['reply_id'].values.tolist()
        score += ndcg_at_k(tmp, len(set(tmp))) / len(uniques)
    score *= 100000
    cv_score += score / nfolds
    
    print('fold#%i: ndcg = %i' % (fold, int(score)))
    fold += 1
print('averaged cv score: ndcg = %i' % (int(cv_score)))

make features...
fitting model...
make prediction...
fold#1: ndcg = 85496
make features...
fitting model...
make prediction...
fold#2: ndcg = 85920
make features...
fitting model...
make prediction...
fold#3: ndcg = 85909
make features...
fitting model...
make prediction...
fold#4: ndcg = 86006
make features...
fitting model...
make prediction...
fold#5: ndcg = 85659
make features...
fitting model...
make prediction...
fold#6: ndcg = 85489
make features...
fitting model...
make prediction...
fold#7: ndcg = 86035
make features...
fitting model...
make prediction...
fold#8: ndcg = 85668
make features...
fitting model...
make prediction...
fold#9: ndcg = 85639
make features...
fitting model...
make prediction...
fold#10: ndcg = 85552
averaged cv score: ndcg = 85737


In [95]:
counters_pipe.fit(np.hstack([train[col] for col in text_cols]))
train_data = np.hstack([counters_pipe.transform(train[col]) for col in text_cols])
test_data = np.hstack([counters_pipe.transform(test[col]) for col in text_cols])

lr.fit(train_data, train['target'])

prediction = pd.DataFrame()
prediction['context_id'] = test[0]
prediction['reply_id'] = test[4]
prediction['rank'] = - lr.predict(test_data)
prediction = prediction.sort_values(by=['context_id', 'rank'])[['context_id', 'reply_id']]

In [96]:
prediction.to_csv(path+'/data/sub_1.tsv',header=None, index=False, sep=' ')

In [236]:
train.to_csv(path+'/data/train_modified.tsv', sep=' ')

In [241]:
train = pd.read_csv(path+'/data/train_modified.tsv', sep=' ', index_col=0)

In [242]:
train.head()

,0,1,2,3,4,5,6,7,rank,target
0,22579918886,"кликни на меня а потом на надпись "" видео - зв...","о , я тебя вижу .","ладно , повесь трубку .",0,не могу .,good,0.875352,3,2.626055
1,22579918886,"кликни на меня а потом на надпись "" видео - зв...","о , я тебя вижу .","ладно , повесь трубку .",1,"нет , звонить буду я .",neutral,0.900968,2,1.801936
2,22579918886,"кликни на меня а потом на надпись "" видео - зв...","о , я тебя вижу .","ладно , повесь трубку .",2,"слушай , я не мог уйти .",bad,0.884320,1,0.884320
3,22579918886,"кликни на меня а потом на надпись "" видео - зв...","о , я тебя вижу .","ладно , повесь трубку .",3,я не прекращу звонить .,good,0.982530,3,2.947591
4,22579918886,"кликни на меня а потом на надпись "" видео - зв...","о , я тебя вижу .","ладно , повесь трубку .",4,я звоню им .,good,0.838054,3,2.514161


In [246]:
import matplotlib.pyplot as plt
import seaborn
%matplotlib inline

In [ ]:
results = {
    'counter_char_ridge':[85700, ],
    'counter_word_ridge':[85738, 84248],
    'tfidf_word_ridge':[85774, 82714],
    'counter_svd_ridge':,
    'counter_tfidf_char_ridge':,
    'counter_tfidf_char_lgbm':,
    'w2v_tfidf_lgbm':,
}